### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Finding the number of different schools in the list
school_list = school_data_complete["school_name"].unique()

#Counting the number of schools
school_count = 0
for school in school_list:
    school_count += 1

#Counting the number of students
total_students = school_data_complete["student_name"].count()

#Summing up the total budgets of the schools (Assumes no two schools have the same budget.)
total_budget = school_data_complete["budget"].unique().sum()
total_budget = "${:,.0f}".format(total_budget)
#Computing and storing the mean reading and math scores
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()

#Creating new df with only passing subsets included
both_pass_df = school_data_complete.loc[(school_data_complete["reading_score"]>=70) & (school_data_complete["math_score"]>=70),["student_name"]]
read_only_df = school_data_complete.loc[school_data_complete["reading_score"]>=70, ["student_name"]]
math_only_df = school_data_complete.loc[school_data_complete["math_score"]>=70, ["student_name"]]

#Counting passing students
both_count =  both_pass_df.count()
read_only = read_only_df.count()
math_only = math_only_df.count()

#Computing the percentage of students who passed for the district
percent_both = int(both_count)/total_students*100
percent_read = int(read_only)/total_students*100
percent_math = int(math_only)/total_students*100

#Formatting Total Students
total_students = "{:,}".format(total_students)

#Creating district summary data frame.
district_summary = pd.DataFrame(
    {"Total Schools": [school_count],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_read_score],
     "% Passing Math": [percent_math],
     "% Passing Reading": [percent_read],
     "% Overall Passing": [percent_both]
    }
)

#Showing the district summary in the output
district_summary.style.hide_index()


Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
15,"39,170","$24,649,428",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#Creating empty lists
school_type = []
total_school_pop = []
school_budget = []
per_student_budget = []
school_avg_math = []
school_avg_read = []
school_per_math = []
school_per_read = []
school_per_over = []

#Looping through subset school data frames and appending data for each school to the correct list.
for school in school_list:
    school_df = school_data_complete[school_data_complete["school_name"]==str(school)]
    school_type_list = school_df["type"].values.tolist()
    school_type.append(school_type_list[0])
    total_school_pop.append(school_df["student_name"].count())
    school_budget_list = school_df["budget"].values.tolist()
    school_budget_item = "${:,.0f}".format(school_budget_list[0])
    school_budget.append(school_budget_item)
    per_student_item = float(school_budget_list[0])/school_df["student_name"].count()
    per_student_item = "${:,.2f}".format(per_student_item)
    per_student_budget.append(per_student_item)
    school_avg_math.append(school_df["math_score"].mean())
    school_avg_read.append(school_df["reading_score"].mean())

    #Creating new df for each school with only passing subsets included
    both_pass_df = school_df.loc[(school_df["reading_score"]>=70) & (school_df["math_score"]>=70),["student_name"]]
    read_only_df = school_df.loc[school_df["reading_score"]>=70, ["student_name"]]
    math_only_df = school_df.loc[school_df["math_score"]>=70, ["student_name"]]

    #Counting passing students
    both_count =  both_pass_df.count()
    read_only = read_only_df.count()
    math_only = math_only_df.count()

    #Computing passing percentages and appending them to the correct list
    school_per_over.append(int(both_count)/school_df["student_name"].count()*100)
    school_per_read.append(int(read_only)/school_df["student_name"].count()*100)
    school_per_math.append(int(math_only)/school_df["student_name"].count()*100)

#Creating a new summary data frame using a dictionary of the school data lists.
school_summary_df = pd.DataFrame(
    {"School Name": school_list,
    "School Type": school_type,
    "Total Students": total_school_pop,
    "Total School Budget": school_budget,
    "Per Student Budget": per_student_budget,
     "Average Math Score": school_avg_math,
     "Average Reading Score": school_avg_read,
     "% Passing Math": school_per_math,
     "% Passing Reading": school_per_read,
     "% Overall Passing": school_per_over})

#Sorting the dataframe alphabetically by school name
school_summary_df.sort_values(by=["School Name"]).style.hide_index()


School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
#Sorting the dataframe by overall pass rate
school_summary_df.sort_values(by=["% Overall Passing"], ascending=False).head().style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
#Sorting the dataframe by overall pass rate
school_summary_df.sort_values(by=["% Overall Passing"]).head().style.hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#Copying the original dataframe
school_df2 = school_data_complete

#Creating empty lists for the math averages
frosh_list = []
soph_list = []
junior_list = []
senior_list = []

#Looping through dataframe by schools and grouping students by grade level
for school in school_list:
    frosh_df = school_df2[(school_df2["grade"]=="9th") & (school_df2["school_name"]==str(school))]
    frosh_list.append(frosh_df["math_score"].mean())
    soph_df = school_df2[(school_df2["grade"]=="10th") & (school_df2["school_name"]==str(school))]
    soph_list.append(soph_df["math_score"].mean())                        
    junior_df = school_df2[(school_df2["grade"]=="11th") & (school_df2["school_name"]==str(school))]
    junior_list.append(junior_df["math_score"].mean())
    senior_df = school_df2[(school_df2["grade"]=="12th") & (school_df2["school_name"]==str(school))]
    senior_list.append(senior_df["math_score"].mean())

#Creating a math summary frame using a dictionary of the grade level lists.
math_summary_df = pd.DataFrame(
    {"School Name": school_list,
    "9th": frosh_list,
    "10th": soph_list,
    "11th": junior_list,
    "12th": senior_list})

#Sorting the dataframe alphabetically by school name
math_summary_df.sort_values(by=["School Name"]).style.hide_index()                              



School Name,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
#Creating empty lists for the reading averages
frosh_list = []
soph_list = []
junior_list = []
senior_list = []

#Looping through dataframe by schools and grouping students by grade level
for school in school_list:
    frosh_df = school_df2[(school_df2["grade"]=="9th") & (school_df2["school_name"]==str(school))]
    frosh_list.append(frosh_df["reading_score"].mean())
    soph_df = school_df2[(school_df2["grade"]=="10th") & (school_df2["school_name"]==str(school))]
    soph_list.append(soph_df["reading_score"].mean())                        
    junior_df = school_df2[(school_df2["grade"]=="11th") & (school_df2["school_name"]==str(school))]
    junior_list.append(junior_df["reading_score"].mean())
    senior_df = school_df2[(school_df2["grade"]=="12th") & (school_df2["school_name"]==str(school))]
    senior_list.append(senior_df["reading_score"].mean())

#Creating a reading summary frame using a dictionary of the grade level lists.
math_summary_df = pd.DataFrame(
    {"School Name": school_list,
    "9th": frosh_list,
    "10th": soph_list,
    "11th": junior_list,
    "12th": senior_list})

#Sorting the dataframe alphabetically by school name
math_summary_df.sort_values(by=["School Name"]).style.hide_index()                              


School Name,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
#Creating empty lists
school_type = []
total_school_pop = []
school_budget = []
per_student_budget = []
school_avg_math = []
school_avg_read = []
school_per_math = []
school_per_read = []
school_per_over = []

#Looping through subset school data frames and appending data for each school to the correct list.
for school in school_list:
    school_df = school_data_complete[school_data_complete["school_name"]==str(school)]
    school_type_list = school_df["type"].values.tolist()
    school_type.append(school_type_list[0])
    total_school_pop.append(school_df["student_name"].count())
    school_budget_list = school_df["budget"].values.tolist()
    school_budget_item = school_budget_list[0]
    school_budget.append(school_budget_item)
    per_student_item = float(school_budget_list[0])/school_df["student_name"].count()
    per_student_budget.append(per_student_item)
    school_avg_math.append(school_df["math_score"].mean())
    school_avg_read.append(school_df["reading_score"].mean())

    #Creating new df for each school with only passing subsets included
    both_pass_df = school_df.loc[(school_df["reading_score"]>=70) & (school_df["math_score"]>=70),["student_name"]]
    read_only_df = school_df.loc[school_df["reading_score"]>=70, ["student_name"]]
    math_only_df = school_df.loc[school_df["math_score"]>=70, ["student_name"]]

    #Counting passing students
    both_count =  both_pass_df.count()
    read_only = read_only_df.count()
    math_only = math_only_df.count()

    #Computing passing percentages and appending them to the correct list
    school_per_over.append(int(both_count)/school_df["student_name"].count()*100)
    school_per_read.append(int(read_only)/school_df["student_name"].count()*100)
    school_per_math.append(int(math_only)/school_df["student_name"].count()*100)

#Creating an unformatted summary data frame using a dictionary of the school data lists.
clean_df = pd.DataFrame(
    {"School Name": school_list,
    "School Type": school_type,
    "Total Students": total_school_pop,
    "Total School Budget": school_budget,
    "Per Student Budget": per_student_budget,
     "Average Math Score": school_avg_math,
     "Average Reading Score": school_avg_read,
     "% Passing Math": school_per_math,
     "% Passing Reading": school_per_read,
     "% Overall Passing": school_per_over})

clean_df.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,89.892107
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455


In [13]:
#Copying the unformatted school summary for use here.
spending_df = clean_df

#Bucket List broken down by per pupil spending
bucket_list = ["<=$585", "$586-630", "$631-645", "$646-680"]

#Creating empty lists for each variable
avg_math = []
avg_read = []
per_math = []
per_read = []
overall = []

#Creating subset dataframe for each spending category
low_df = spending_df[spending_df["Per Student Budget"]<=585.0]
lowmed_df = spending_df[(spending_df["Per Student Budget"]>585.0) & (spending_df["Per Student Budget"]<=630.00)]
med_df = spending_df[(spending_df["Per Student Budget"]>630.0) & (spending_df["Per Student Budget"]<=645.0)]
high_df = spending_df[(spending_df["Per Student Budget"]>645.0) & (spending_df["Per Student Budget"]<700.0)]

#Computing the average math score for the low ppe and appending it to a list
low_df.loc[:,"total_math_scores"]=low_df["Average Math Score"]*low_df["Total Students"]
avg_math_item = low_df["total_math_scores"].sum()/low_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for the low ppe and appending it to a list
low_df.loc[:,"total_reading_scores"]=low_df["Average Reading Score"]*low_df["Total Students"]
avg_read_item = low_df["total_reading_scores"].sum()/low_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for the low ppe and appending it to a list
low_df.loc[:,"total_math_pass"]=low_df["% Passing Math"]*low_df["Total Students"]/100
math_per_item = low_df["total_math_pass"].sum()/low_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for the low ppe and appending it to a list
low_df.loc[:,"total_read_pass"]=low_df["% Passing Reading"]*low_df["Total Students"]/100
read_per_item = low_df["total_read_pass"].sum()/low_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for the low ppe and appending it to a list
low_df.loc[:,"total_overall_pass"]=low_df["% Overall Passing"]*low_df["Total Students"]/100
overall_per_item = low_df["total_overall_pass"].sum()/low_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Computing the average math score for the lowmed ppe and appending it to a list
lowmed_df.loc[:,"total_math_scores"]=lowmed_df["Average Math Score"]*lowmed_df["Total Students"]
avg_math_item = lowmed_df["total_math_scores"].sum()/lowmed_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for the lowmed ppe and appending it to a list
lowmed_df.loc[:,"total_reading_scores"]=lowmed_df["Average Reading Score"]*lowmed_df["Total Students"]
avg_read_item = lowmed_df["total_reading_scores"].sum()/lowmed_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for the lowmed ppe and appending it to a list
lowmed_df.loc[:,"total_math_pass"]=lowmed_df["% Passing Math"]*lowmed_df["Total Students"]/100
math_per_item = lowmed_df["total_math_pass"].sum()/lowmed_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for the lowmed ppe and appending it to a list
lowmed_df.loc[:,"total_read_pass"]=lowmed_df["% Passing Reading"]*lowmed_df["Total Students"]/100
read_per_item = lowmed_df["total_read_pass"].sum()/lowmed_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for the lowmed ppe and appending it to a list
lowmed_df.loc[:,"total_overall_pass"]=lowmed_df["% Overall Passing"]*lowmed_df["Total Students"]/100
overall_per_item = lowmed_df["total_overall_pass"].sum()/lowmed_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Computing the average math score for the med ppe and appending it to a list
med_df.loc[:,"total_math_scores"]=med_df["Average Math Score"]*med_df["Total Students"]
avg_math_item = lowmed_df["total_math_scores"].sum()/lowmed_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for the med ppe and appending it to a list
med_df.loc[:,"total_reading_scores"]=med_df["Average Reading Score"]*med_df["Total Students"]
avg_read_item = med_df["total_reading_scores"].sum()/med_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for the med ppe and appending it to a list
med_df.loc[:,"total_math_pass"]=med_df["% Passing Math"]*med_df["Total Students"]/100
math_per_item = med_df["total_math_pass"].sum()/med_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for the med ppe and appending it to a list
med_df.loc[:,"total_read_pass"]=med_df["% Passing Reading"]*med_df["Total Students"]/100
read_per_item = med_df["total_read_pass"].sum()/med_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for the med ppe and appending it to a list
med_df.loc[:,"total_overall_pass"]=med_df["% Overall Passing"]*med_df["Total Students"]/100
overall_per_item = med_df["total_overall_pass"].sum()/med_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Computing the average math score for the high ppe and appending it to a list
high_df.loc[:,"total_math_scores"]=high_df["Average Math Score"]*high_df["Total Students"]
avg_math_item = high_df["total_math_scores"].sum()/high_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for the high ppe and appending it to a list
high_df.loc[:,"total_reading_scores"]=high_df["Average Reading Score"]*high_df["Total Students"]
avg_read_item = high_df["total_reading_scores"].sum()/high_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for the high ppe and appending it to a list
high_df.loc[:,"total_math_pass"]=high_df["% Passing Math"]*high_df["Total Students"]/100
math_per_item = high_df["total_math_pass"].sum()/high_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for the high ppe and appending it to a list
high_df.loc[:,"total_read_pass"]=high_df["% Passing Reading"]*high_df["Total Students"]/100
read_per_item = high_df["total_read_pass"].sum()/high_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for the high ppe and appending it to a list
high_df.loc[:,"total_overall_pass"]=high_df["% Overall Passing"]*high_df["Total Students"]/100
overall_per_item = high_df["total_overall_pass"].sum()/high_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Creating a summary dataframe
spending_summary_df = pd.DataFrame(
    {"Per Student Spending": bucket_list,
    "Average Math Scores": avg_math,
    "Average Reading Scores": avg_read,
     "% Passing Math": per_math,
     "% Passing Reading": per_read,
     "% Overall Passing": overall})

spending_summary_df.style.hide_index()

Per Student Spending,Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Overall Passing
<=$585,83.363065,83.964039,93.702889,96.686558,90.640704
$586-630,79.982873,82.312643,79.109851,88.513145,70.939239
$631-645,79.982873,81.301007,70.623565,82.600247,58.841194
$646-680,77.049297,81.005604,66.230813,81.109397,53.528791


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
#Saving the school_summary_df as size_df
size_df=spending_df

#Creating list of school size buckets
size_bucket_list = ['0 to 1500', '1501-2500', '2501-4000', '4001-5000']

#Creating empty lists for each variable
avg_math = []
avg_read = []
per_math = []
per_read = []
overall = []

#Creating subset dataframe for each school size category
low_df = size_df[size_df["Total Students"]<=1500]
#Computing the average math score for this school size and appending it to a list
low_df.loc[:,"total_math_scores"]=low_df["Average Math Score"]*low_df["Total Students"]
avg_math_item = low_df["total_math_scores"].sum()/low_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for this school size and appending it to a list
low_df.loc[:,"total_reading_scores"]=low_df["Average Reading Score"]*low_df["Total Students"]
avg_read_item = low_df["total_reading_scores"].sum()/low_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for this school size and appending it to a list
low_df.loc[:,"total_math_pass"]=low_df["% Passing Math"]*low_df["Total Students"]/100
math_per_item = low_df["total_math_pass"].sum()/low_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for this school size and appending it to a list
low_df.loc[:,"total_read_pass"]=low_df["% Passing Reading"]*low_df["Total Students"]/100
read_per_item = low_df["total_read_pass"].sum()/low_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for this school size and appending it to a list
low_df.loc[:,"total_overall_pass"]=low_df["% Overall Passing"]*low_df["Total Students"]/100
overall_per_item = low_df["total_overall_pass"].sum()/low_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Repeating for the lowmed bucket
lowmed_df = size_df[(size_df["Total Students"]>1500) & (size_df["Total Students"]<=2500)]
#Computing the average math score for this school size and appending it to a list
lowmed_df.loc[:,"total_math_scores"]=lowmed_df["Average Math Score"]*lowmed_df["Total Students"]
avg_math_item = lowmed_df["total_math_scores"].sum()/lowmed_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for this school size and appending it to a list
lowmed_df.loc[:,"total_reading_scores"]=lowmed_df["Average Reading Score"]*lowmed_df["Total Students"]
avg_read_item = lowmed_df["total_reading_scores"].sum()/lowmed_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for this school size and appending it to a list
lowmed_df.loc[:,"total_math_pass"]=lowmed_df["% Passing Math"]*lowmed_df["Total Students"]/100
math_per_item = lowmed_df["total_math_pass"].sum()/lowmed_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for this school size and appending it to a list
lowmed_df.loc[:,"total_read_pass"]=lowmed_df["% Passing Reading"]*lowmed_df["Total Students"]/100
read_per_item = lowmed_df["total_read_pass"].sum()/lowmed_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for this school size and appending it to a list
lowmed_df.loc[:,"total_overall_pass"]=lowmed_df["% Overall Passing"]*lowmed_df["Total Students"]/100
overall_per_item = lowmed_df["total_overall_pass"].sum()/lowmed_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Repeating for the med bucket
med_df = size_df[(size_df["Total Students"]>2500) & (size_df["Total Students"]<=4000)]
#Computing the average math score for this school size and appending it to a list
med_df.loc[:,"total_math_scores"]=med_df["Average Math Score"]*med_df["Total Students"]
avg_math_item = lowmed_df["total_math_scores"].sum()/lowmed_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for this school size and appending it to a list
med_df.loc[:,"total_reading_scores"]=med_df["Average Reading Score"]*med_df["Total Students"]
avg_read_item = med_df["total_reading_scores"].sum()/med_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for this school size and appending it to a list
med_df.loc[:,"total_math_pass"]=med_df["% Passing Math"]*med_df["Total Students"]/100
math_per_item = med_df["total_math_pass"].sum()/med_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for this school size and appending it to a list
med_df.loc[:,"total_read_pass"]=med_df["% Passing Reading"]*med_df["Total Students"]/100
read_per_item = med_df["total_read_pass"].sum()/med_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for this school size and appending it to a list
med_df.loc[:,"total_overall_pass"]=med_df["% Overall Passing"]*med_df["Total Students"]/100
overall_per_item = med_df["total_overall_pass"].sum()/med_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Repeating for the high bucket
high_df = size_df[(size_df["Total Students"]>4000) & (size_df["Total Students"]<=5000)]
#Computing the average math score for this spending group and appending it to a list
high_df.loc[:,"total_math_scores"]=high_df["Average Math Score"]*high_df["Total Students"]
avg_math_item = high_df["total_math_scores"].sum()/high_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for this school size and appending it to a list
high_df.loc[:,"total_reading_scores"]=high_df["Average Reading Score"]*high_df["Total Students"]
avg_read_item = high_df["total_reading_scores"].sum()/high_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for this school size and appending it to a list
high_df.loc[:,"total_math_pass"]=high_df["% Passing Math"]*high_df["Total Students"]/100
math_per_item = high_df["total_math_pass"].sum()/high_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for this school size and appending it to a list
high_df.loc[:,"total_read_pass"]=high_df["% Passing Reading"]*high_df["Total Students"]/100
read_per_item = high_df["total_read_pass"].sum()/high_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for this school size and appending it to a list
high_df.loc[:,"total_overall_pass"]=high_df["% Overall Passing"]*high_df["Total Students"]/100
overall_per_item = high_df["total_overall_pass"].sum()/high_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Creating a summary dataframe
size_summary_df = pd.DataFrame(
    {"School Size (Students)": size_bucket_list,
    "Average Math Scores": avg_math,
    "Average Reading Scores": avg_read,
     "% Passing Math": per_math,
     "% Passing Reading": per_read,
     "% Overall Passing": overall})

size_summary_df.style.hide_index()

School Size (Students),Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Overall Passing
0 to 1500,83.583479,83.893245,93.664683,96.604830,90.374519
1501-2500,83.351933,83.905751,93.713184,96.658456,90.617972
2501-4000,83.351933,80.943113,66.542368,80.395113,53.443351
4001-5000,77.134219,80.979474,66.497356,81.352630,53.917339


## Scores by School Type

* Perform the same operations as above, based on school type

In [11]:
#Saving the school_summary_df as type_df
type_df=spending_df

#Creating list of school size buckets
type_bucket_list = ['District', 'Charter']

#Creating empty lists for each variable
avg_math = []
avg_read = []
per_math = []
per_read = []
overall = []

#Creating subset dataframe for each school type category
district_df = type_df[type_df["School Type"]=="District"]
#Computing the average math score for this spending group and appending it to a list
district_df.loc[:,"total_math_scores"]=district_df["Average Math Score"]*district_df["Total Students"]
avg_math_item = district_df["total_math_scores"].sum()/district_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for this spending group and appending it to a list
district_df.loc[:,"total_reading_scores"]=district_df["Average Reading Score"]*district_df["Total Students"]
avg_read_item = district_df["total_reading_scores"].sum()/district_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for this spending group and appending it to a list
district_df.loc[:,"total_math_pass"]=district_df["% Passing Math"]*district_df["Total Students"]/100
math_per_item = district_df["total_math_pass"].sum()/district_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for this spending group and appending it to a list
district_df.loc[:,"total_read_pass"]=district_df["% Passing Reading"]*district_df["Total Students"]/100
read_per_item = district_df["total_read_pass"].sum()/district_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for this spending group and appending it to a list
district_df.loc[:,"total_overall_pass"]=district_df["% Overall Passing"]*district_df["Total Students"]/100
overall_per_item = district_df["total_overall_pass"].sum()/district_df["Total Students"].sum()*100
overall.append(overall_per_item)

#Repeating for the charter school
#Creating subset dataframe for each school type category
charter_df = type_df[type_df["School Type"]=="Charter"]
#Computing the average math score for this spending group and appending it to a list
charter_df.loc[:,"total_math_scores"]=charter_df["Average Math Score"]*charter_df["Total Students"]
avg_math_item = charter_df["total_math_scores"].sum()/charter_df["Total Students"].sum()
avg_math.append(avg_math_item)
#Computing the average reading score for this spending group and appending it to a list
charter_df.loc[:,"total_reading_scores"]=charter_df["Average Reading Score"]*charter_df["Total Students"]
avg_read_item = charter_df["total_reading_scores"].sum()/charter_df["Total Students"].sum()
avg_read.append(avg_read_item)
#Computing the math percent pass rate for this spending group and appending it to a list
charter_df.loc[:,"total_math_pass"]=charter_df["% Passing Math"]*charter_df["Total Students"]/100
math_per_item = charter_df["total_math_pass"].sum()/charter_df["Total Students"].sum()*100
per_math.append(math_per_item)
#Computing the reading percent pass rate for this spending group and appending it to a list
charter_df.loc[:,"total_read_pass"]=charter_df["% Passing Reading"]*charter_df["Total Students"]/100
read_per_item = charter_df["total_read_pass"].sum()/charter_df["Total Students"].sum()*100
per_read.append(read_per_item)
#Computing the overall pass rate for this spending group and appending it to a list
charter_df.loc[:,"total_overall_pass"]=charter_df["% Overall Passing"]*charter_df["Total Students"]/100
overall_per_item = charter_df["total_overall_pass"].sum()/charter_df["Total Students"].sum()*100
overall.append(overall_per_item)


#Creating a summary dataframe
type_summary_df = pd.DataFrame(
    {"School Type": type_bucket_list,
    "Average Math Scores": avg_math,
    "Average Reading Scores": avg_read,
     "% Passing Math": per_math,
     "% Passing Reading": per_read,
     "% Overall Passing": overall})

type_summary_df.style.hide_index()

School Type,Average Math Scores,Average Reading Scores,% Passing Math,% Passing Reading,% Overall Passing
District,76.987026,80.962485,66.518387,80.905249,53.695878
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
